# Appendix

## Tree of Parzen Estimators
### General Idea
Typically, researchers are confronted with the problem that it takes considerable amounts of time when trying to evaluate a cost function. This problem is especially aggravating, when the optimization of that costly objective function is performed over a large grid of hyperparameters. 

The general idea of the tree of Parzen Estimators is remarkably simple. When we evaluate the objective function on multiple sets of hyperparameters $x$, we can always fix a threshold $y^\star$ such that some evaluations are below that threshold, and some lie above. Effectively this means, that we have split our different evaluations into two parts: a good one, where the evaluated objective function lies below $y^\star$ and a second part that leads to a worse loss larger than $y^\star$, which is bad in our case.

Mathematically speaking, we can split the conditional densities of our hyperparameters $x$ into two parts, such that

\begin{equation}
p(x|y) = \begin{cases}
\color{blue}{l(x)} \text{ if } y<y^\star \\
\color{red}{g(x)} \text{ if } y \geq y^\star.
\end{cases}
\end{equation}

This means, that when we want to consider a new set of hyperparamters, we would like to draw it from the distribution that seems to produce lower values for our objective function, i.e. we want to draw from $\color{blue}{l(x)}$ \[[cp. Bergstra et al 2011](https://papers.nips.cc/paper/4443-algorithms-for-hyper-parameter-optimization.pdf)\]. 

![title](pictures/tpe.png)
\[[cp. Koehrsen 2018](https://towardsdatascience.com/a-conceptual-explanation-of-bayesian-model-based-hyperparameter-optimization-for-machine-learning-b8172278050f)\]

In the figure above we see an (artificial) example from evaluating the loss function of a Random Forest, where $x$ is simply the number of estimators we wish to use. As expected, we see that, the higher the number of trees we include in our Random Forest, the lower the loss we can achieve. When we fix $y^\star=120$ we can separate two subsets from all the values of $x$ that we tried. When plotting their separate Kernel-Density estimates the red density corresponds to the values of $x$ that led to a high loss, whereas the blue distribution is formed by the values of $x$ which led to a low loss. Given that knowledge, when one is to choose a value of $x$ for the next evaluation common sense, and the Tree-structured Parzen Estimator approach encourage us to consider a value that is similar to the blue dots. Therefore we sample from $\color{blue}{l(x)}$, since these values seem to have worked in the previous iterations.

### Mathematical Derivation

As stated, the general idea is remarkably simple. However, the mathematical derivation is a little more convoluted. Formally, the Tree-structured Parzen Estimator's goal is to maximize the Expected Information gain $EI_{y^\star}$ for some fixed $y^\star$


$$
EI_{y^\star}(x) = \int\limits_{-\infty}^{y^\star} (y^\star-y)p(y|x)dy.
$$


The expected information gain is a function of the hyperparameters, where we try to integrate out all interesting, values for y, i.e. those which are smaller than $y^\star$. We can also use Bayes' formula to find a different expression for $p(y|x)$

$$
p(y|x) = \frac{p(x|y)p(y)}{p(x)}.
$$

But before we continue, it will become useful to note some things about the unconditional density of $p(x)$


\begin{align}
p(x) &= \int p(x, y) dy \\
     &= \int p(x|y)p(y) dy \\
     &= \int\limits_{\color{blue}{-\infty}}^{\color{blue}{y^\star}}\color{blue}{p(x|y)}p(y)dy +
        \int\limits^{\color{red}{\infty}}_{\color{red}{y^\star}}\color{red}{p(x|y)}p(y)dy \\
     &= \color{blue}{l(x)} \int\limits_{-\infty}^{y^\star}p(y)dy +
        \color{red}{g(x)} \int\limits^{\infty}_{y^\star}p(y)dy \\
     &= l(x) \underbrace{P(Y<y^\star)}_{:=\gamma} + g(x)    \underbrace{P(Y \geq y^\star)}_{:=1-\gamma} \\
     &= \gamma l(x) + (1-\gamma)g(x)
\end{align}
\[[cp. Bergstra et al 2011](https://papers.nips.cc/paper/4443-algorithms-for-hyper-parameter-optimization.pdf)\]

What does this all mean? First, we can get the unconditional density of $x$ if we integrate out the influence of $y$ from the joint density $p(x, y)$. Second, we can replace the joint density with an expression we get from the definition of the conditional density of 

$$p(x|y) = \frac{p(x, y)}{p(y)} \Leftrightarrow p(x, y) = p(x|y)\ p(y).$$

Third, now that we have an expression that contains $p(x|y)$ we can make use of our definition from the beginning, i.e. we can split the integral into the '$\color{blue}{good}$' and '$\color{red}{bad}$' parts of the real line. And since $\color{blue}{l(x)}$ and $\color{red}{g(x)}$ do not depend on $y$, we can pull them out of the integral in the fourth line. This leaves us with integrals over the unconditional density of $y$ which is just how probabilities are defined. Summing up, this means we can describe the unconditional distribution of the hyperparameters as a weighted sum, where the weights are the probabilities of lying below or above the threshold $y^\star$. 

Coming back to our problem: we want to further investigate the information gain

\begin{align}
EI_{y^\star}(x) &= \int\limits_{-\infty}^{y^\star} (y^\star-y)p(y|x)dy \\
  &= \int\limits_{\color{blue}{-\infty}}^{\color{blue}{y^\star}}(y^\star-y) \frac{\color{blue}{p(x|y)}p(y)}{p(x)}dy
  = \frac{\color{blue}{l(x)}}{p(x)} \int\limits_{-\infty}^{y^\star}(y^\star-y) p(y)dy \\
  &= \frac{l(x)}{p(x)} \left[ y^\star \underbrace{\int\limits_{-\infty}^{y^\star} p(y)dy}_{P(Y < y^\star) = \gamma} - \int\limits_{-\infty}^{y^\star} y \cdot p(y)dy \right]
  = \frac{l(x)\gamma y^\star - l(x)\int_{-\infty}^{y^\star} y \cdot p(y) dy}{l(x) \gamma + (1-\gamma)g(x)} 
  \propto \color{orange}{\left[\gamma + (1-\gamma) \frac{g(x)}{l(x)}\right]^{-1}}
\end{align}
\[[cp. Bergstra et al 2011](https://papers.nips.cc/paper/4443-algorithms-for-hyper-parameter-optimization.pdf)\]

We start by applying Bayes rule to the conditional density of y $p(y|x)$ inside the interval and we note that this leaves us with an expression which is similar to the '$\color{blue}{good}$' part of the conditional density of $x$ given $y$. We pull all terms that do not depend on $y$ out of the integral and multiply out the parenthesis inside it. After multiplying we note that we can replace the integral over the unconditional density of $y$ with the probability that $P(Y<y^\star):= \gamma$ and we note that we can replace $p(x)$ by the expression we derived previously. Unfortunately, we cannot further simplify the remaining integral, even if it looks like an expected value (in it's limits it is missing one half of the real line). Thus we can only factor out $l(x)$ in the numerator and denominator of the second last step such that it - almost - cancels out. Nonetheless, we can state that the numerator will just be a constant which means that the Expected information gain is proportional to the orange expression. This means that, as we outlined in the intuitive explanation, the Expected Information grows if we are more likely to sample from the '$\color{blue}{good}$' density $\color{blue}{l(x)}$ than the '$\color{red}{bad}$' density $\color{red}{g(x)}$ \[[cp. Koehrsen 2018](https://towardsdatascience.com/a-conceptual-explanation-of-bayesian-model-based-hyperparameter-optimization-for-machine-learning-b8172278050f)\].